### Loading in my Libraries

In [45]:
import pandas as pd
import numpy as np
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# #1

### Loading in my DF

In [137]:
df=pd.read_csv('/Users/thomas/Downloads/enigma test (1).csv')

### #1-1

In [138]:
def clean_text(text):
    return text.replace('\t'," ").replace('\n',' ').replace('  ',' ')
# Removing tabs, new lines, and double spaces
df['bio']=df['bio'].apply(lambda x: clean_text(x))

### #1.2

In [139]:
state_abbreviations=pd.read_csv('/Users/thomas/Downloads/enigma state_abbreviations.csv')

In [140]:
# Creating dictionary object of the state abbreviations and the full names
state_abbreviations.index=state_abbreviations['state_abbr']
del state_abbreviations['state_abbr']
states_dict=state_abbreviations.to_dict()['state_name']
# Using dict to convert abbreviations
df['state']=df['state'].apply(lambda x: states_dict[x])

### #1.3

In [141]:
# We can see if the text is datetime compatible; if it is, we will also need to check completeness, thus the '/',...
def check_date_completeness(text):
    try: 
        pd.to_datetime(text)
    except:
        return 0
    if '/' in text:
        if len(text.split('/'))!=3:
            return 0
    elif '-' in text:
        if len(text.split('-'))!=3:
            return 0
    elif ',' in text:
        if len(text.split(',')[0].split())!=2:
            return 0
    else:
        if len(text.split())!=3:
            return 0
    return 1

In [142]:
# Check Completeness; if incomplete, put into new column
df['start_date_description']=df['start_date'].apply(lambda x: x if check_date_completeness(x)==0 else "")
# If details column holds an object, remove the corresponding object in the date column
for row in df.index:
    if df.loc[row,'start_date']==df.loc[row,'start_date_description']:
        df.loc[row,'start_date']=""
# Convert to datetime; automatic iso
df['start_date']=pd.to_datetime(df['start_date'])
# Save to csv
df.to_csv('/Users/thomas/Downloads/enigma solutions.csv')

# #2

In [252]:
import requests
import BeautifulSoup
import numpy as np
import pandas as pd

In [246]:
company_data=[]
column_info=[]
for i in range(1,11):
    # baseurl and pageurl seperated due to the urls of the individual hrefs
    baseurl='http://data-interview.enigmalabs.org'
    pageurl='/companies/?page=%s'%(i)
    # getting the html
    r=requests.get(baseurl+pageurl)
    soup=BeautifulSoup.BeautifulSoup(r.text)
    # getting the correct hrefs from the html, and replacing ' ' with %20
    links=[x.a for x in soup.findAll('tr')][1:]
    links=[x['href'].replace(' ','%20') for x in links]
    for link in links:
        # going to href locations
        info_url=baseurl+link
        info_r=requests.get(info_url)
        info_soup=BeautifulSoup.BeautifulSoup(info_r.text)
        # pulling in all info, and then selecting the correct info
        company_data.append([y[1] for y in np.reshape([x.text for x in info_soup.findAll('td')],(-1,2))])
        column_info=[y[0] for y in np.reshape([x.text for x in info_soup.findAll('td')],(-1,2))]
# converting to df, and then saving df as a json
json_df=pd.DataFrame(company_data,columns=column_info)
json_df.to_json('/Users/thomas/Downloads/enigma json solutions.json')